In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM
from keras.models import Sequential, load_model

# === Step 0: Load Data ===
df = pd.read_csv("synthetic_trashcan_fill_levels2.csv")

# === Step 1: Select One Trashcan ===
df = df.drop(columns=["edgeID"])
trashcan_id = df['trashcanID'].iloc[0]
trashcan_data = df[df['trashcanID'] == trashcan_id].drop(columns=["trashcanID"]).values.flatten()

# === Step 2: Normalize and Create Sequences ===
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(trashcan_data.reshape(-1, 1))

# Use past 10 days to predict next
def create_sequences(data, window_size=10):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

window_size = 10
X, y = create_sequences(scaled_data, window_size)

X = X.reshape((X.shape[0], X.shape[1], 1))

# === Step 3: Split into train/test ===
split = int(len(X) * 0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# === Step 4: Train LSTM ===
model = Sequential([
    LSTM(50, activation='relu', input_shape=(window_size, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, verbose=0)

# === Step 5: Predict Today's Level ===
last_sequence = scaled_data[-window_size:]  # last 10 days
last_sequence = last_sequence.reshape((1, window_size, 1))
pred_scaled = model.predict(last_sequence)[0][0]
predicted_fill = scaler.inverse_transform([[pred_scaled]])[0][0]

# === Step 6: Check Actual Today Fill Level (last day in history) ===
actual_today = trashcan_data[-1]

# === Step 7: Decision Logic ===
if actual_today >= 80 or predicted_fill >= 80:
    print(f"🟠 Route Required for Trashcan: {trashcan_id}")
    print(f"  🔹 Actual Fill: {actual_today:.2f}%")
    print(f"  🔹 Predicted Fill: {predicted_fill:.2f}%")
else:
    print(f"🟢 No route needed for {trashcan_id}.")
    print(f"  🔹 Actual Fill: {actual_today:.2f}%")
    print(f"  🔹 Predicted Fill: {predicted_fill:.2f}%")

# === Step 8: Evaluation ===
pred_test = model.predict(X_test)
pred_test_unscaled = scaler.inverse_transform(pred_test)
y_test_unscaled = scaler.inverse_transform(y_test.reshape(-1, 1))

mse = mean_squared_error(y_test_unscaled, pred_test_unscaled)
print(f"\n📊 model mean squared error on test set: {mse:.2f}")


c:\Users\schak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
🟢 No route needed for T1.
  🔹 Actual Fill: 73.00%
  🔹 Predicted Fill: 73.38%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

📊 model mean squared error on test set: 199.28
